# Objective

Predict if a property is a good investment based on financial and physical attributes.

<b>Target:</b>

A binary variable indicating good (1) or bad (0) investment. Define this based on criteria such as a high rent-to-price ratio or a favorable market estimate compared to the listed price.


In [634]:
import pandas as pd

# Data exploration


In [635]:
real_estate_data = pd.read_csv("./data/real_estate_data.csv")
real_estate_data.describe()

# Find any missing data in the dataset.
missing_values = real_estate_data.isnull().sum()

missing_values

State                0
City                 0
Street               0
Zipcode              0
Bedroom             14
Bathroom            34
Area                 0
PPSq                 0
LotArea            902
MarketEstimate    7236
RentEstimate      5976
Latitude             0
Longitude            0
ListedPrice          0
dtype: int64

In [636]:
# See how much data there is with no null values. This can help us determine if we can simply remove all of the nulls from the dataset.

len(real_estate_data.dropna()), real_estate_data.dropna().isnull().sum()

(14853,
 State             0
 City              0
 Street            0
 Zipcode           0
 Bedroom           0
 Bathroom          0
 Area              0
 PPSq              0
 LotArea           0
 MarketEstimate    0
 RentEstimate      0
 Latitude          0
 Longitude         0
 ListedPrice       0
 dtype: int64)

In [637]:
# Get the average difference of List to Estimate by state.

no_null_market_estimate = real_estate_data[real_estate_data["MarketEstimate"].notnull()]

avg_market_minus_list = (
    real_estate_data["MarketEstimate"] - real_estate_data["ListedPrice"]
).median()
avg_market_minus_list

-1600.0

# Data preprocessing


In [638]:
# Remove all rows without a Rent Estimate as this value is vital to my analysis.

real_estate_data = real_estate_data[real_estate_data["RentEstimate"].notnull()]

len(real_estate_data), real_estate_data.isnull().sum()

(16705,
 State                0
 City                 0
 Street               0
 Zipcode              0
 Bedroom              9
 Bathroom            24
 Area                 0
 PPSq                 0
 LotArea            611
 MarketEstimate    1308
 RentEstimate         0
 Latitude             0
 Longitude            0
 ListedPrice          0
 dtype: int64)

In [639]:
# Update market estimates with the average difference between
for index, row in real_estate_data.iterrows():
    if pd.isna(row["MarketEstimate"]):
        real_estate_data.at[index, "MarketEstimate"] = (
            row["ListedPrice"] + avg_market_minus_list
        )

real_estate_data.isnull().sum()

State               0
City                0
Street              0
Zipcode             0
Bedroom             9
Bathroom           24
Area                0
PPSq                0
LotArea           611
MarketEstimate      0
RentEstimate        0
Latitude            0
Longitude           0
ListedPrice         0
dtype: int64

In [640]:
# Populate bed, bath, and lot area based on the median area for the given state.


def get_median_for_row(row: pd.Series, df: pd.DataFrame, area_offset=200):
    area = row["Area"]
    filtered_df = df[df["State"].eq(row["State"])].query(
        f"Area <= {area + area_offset} or Area >= {area - area_offset}"
    )
    median_bed = filtered_df["Bedroom"].median()
    median_bath = filtered_df["Bathroom"].median()
    median_lot = filtered_df["LotArea"].median()
    return median_bed, median_bath, median_lot


for index, row in real_estate_data.iterrows():
    if pd.isna(row["Bedroom"]) or pd.isna(row["Bathroom"]) or pd.isna(row["LotArea"]):
        if pd.isna(row["Bedroom"]):
            median_bed, _, _ = get_median_for_row(row, real_estate_data)
            real_estate_data.at[index, "Bedroom"] = median_bed
        if pd.isna(row["Bathroom"]):
            _, median_bath, _ = get_median_for_row(row, real_estate_data)
            real_estate_data.at[index, "Bathroom"] = median_bath
        if pd.isna(row["LotArea"]):
            _, _, median_lot = get_median_for_row(row, real_estate_data)
            real_estate_data.at[index, "LotArea"] = median_lot

real_estate_data.isnull().sum()

State             0
City              0
Street            0
Zipcode           0
Bedroom           0
Bathroom          0
Area              0
PPSq              0
LotArea           0
MarketEstimate    0
RentEstimate      0
Latitude          0
Longitude         0
ListedPrice       0
dtype: int64

In [641]:
# Convert zipcodes to string data as the number representation does not provide much value.


def zip_to_zip_str(numeric_zip: float):
    rounded_zip = int(numeric_zip)
    return str(rounded_zip).zfill(5)


real_estate_data["Zipcode"] = real_estate_data["Zipcode"].apply(zip_to_zip_str)

real_estate_data["Zipcode"].unique()

array(['36571', '35043', '35811', ..., '82649', '83112', '82932'],
      dtype=object)

In [642]:
from src.const import property_tax_rates

# Calculating the additional features:

real_estate_data["AnnualPropertyTaxEstimate"] = real_estate_data[
    "MarketEstimate"
] * real_estate_data["State"].map(property_tax_rates)


# Estimated Monthly Mortgage - assuming a 30-year fixed mortgage at 6% interest rate.
interest_rate = 0.06 / 12
loan_term = 30 * 12  # 30 years fixed rate loan.
real_estate_data["MonthlyMortgageEstimate"] = (
    real_estate_data["ListedPrice"] * interest_rate * (1 + interest_rate) ** loan_term
) / ((1 + interest_rate) ** loan_term - 1)

real_estate_data[
    [
        "ListedPrice",
        "MarketEstimate",
        "RentEstimate",
        "AnnualPropertyTaxEstimate",
        "MonthlyMortgageEstimate",
    ]
].head()

,ListedPrice,MarketEstimate,RentEstimate,AnnualPropertyTaxEstimate,MonthlyMortgageEstimate
0,239900.0,240600.0,1599.0,986.46,1438.321710
3,335000.0,336200.0,1932.0,1378.42,2008.494259
4,250000.0,222700.0,1679.0,913.07,1498.876313
5,151000.0,150500.0,1385.0,617.05,905.321293
6,239000.0,238400.0,2125.0,977.44,1432.925755


In [643]:
# For this exercise I am going to populate the "GoodInvestment" field with 1 when the Rent price exceeds the annual cost of the home.

real_estate_data["AnnualCost"] = (
    real_estate_data["MonthlyMortgageEstimate"] * 12
) + real_estate_data["AnnualPropertyTaxEstimate"]
real_estate_data["AnnualIncome"] = real_estate_data["RentEstimate"] * 12

real_estate_data["GoodInvestment"] = real_estate_data.apply(
    lambda row: 1 if row["AnnualIncome"] > row["AnnualCost"] else 0, axis=1
)

len(real_estate_data), real_estate_data["GoodInvestment"].value_counts()

(16705,
 GoodInvestment
 0    10523
 1     6182
 Name: count, dtype: int64)

In [644]:
# Create function which creates testing and training data drops fields that should not be included in the analysis

from sklearn.model_selection import train_test_split


def train_test(df: pd.DataFrame):
    X = df.drop(
        [
            "GoodInvestment",
            "AnnualCost",
            "AnnualIncome",
            "RentEstimate",
            "MonthlyMortgageEstimate",
            "AnnualPropertyTaxEstimate",
        ],
        axis=1,
    )

    y = df["GoodInvestment"]

    return train_test_split(X, y, test_size=0.2, random_state=42)

# Decision Tree


In [645]:
# Decisions trees sometimes perform better when continuous data data is binned. We will bin values like areas, beds, bathrooms, and prices.

area_bins = [
    0,
    1000,
    1500,
    2000,
    2500,
    3000,
    3500,
    4000,
    5000,
    7500,
    10000,
    real_estate_data["Area"].max() + 1,
]
area_labels = [
    "0-1000 sqft",
    "1000-1500 sqft",
    "1500-2000 sqft",
    "2000-2500 sqft",
    "2500-3000 sqft",
    "3000-3500 sqft",
    "3500-4000 sqft",
    "4000-5000 sqft",
    "5000-7500 sqft",
    "7500-10000 sqft",
    "10000+ sqft",
]

price_bins = [
    0,
    150_000,
    200_000,
    250_000,
    300_000,
    350_000,
    400_000,
    450_000,
    500_000,
    600_000,
    750_000,
    1_000_000,
    2_500_000,
    5_000_000,
    10_000_000,
    real_estate_data["ListedPrice"].max() + 1,
]
price_labels = [
    "0-150k",
    "150k-200k",
    "200k-250k",
    "250k-300k",
    "300k-350k",
    "350k-400k",
    "400k-450k",
    "450k-500k",
    "500k-600k",
    "600k-750k",
    "750k-1m",
    "1m-2m",
    "2.5m-5m",
    "5m-10m",
    "10m+",
]

lot_bins = [
    0,
    0.11,
    0.17,
    0.23,
    0.35,
    0.5,
    1,
    1.5,
    3,
    10,
    real_estate_data["LotArea"].max() + 1,
]
lot_labels = [
    "0-0.11 acres",
    "0.11-0.17 acres",
    "0.17-0.23 acres",
    "0.23-0.35 acres",
    "0.35-0.5 acres",
    "0.5-1 acres",
    "1-1.5 acres",
    "1.5-3 acres",
    "3-10 acres",
    "10+ acres",
]

bed_bins = [0] + real_estate_data["Bathroom"].sort_values().unique()
bed_labels = [str(bed) for bed in bed_bins]
bed_labels.pop(0)

bath_bins = [0] + real_estate_data["Bedroom"].sort_values().unique()
bath_labels = [str(bath) for bath in bath_bins]
bath_labels.pop(0)

price_bins, price_labels, area_bins, area_labels, lot_bins, lot_labels, bed_bins, bed_labels, bath_bins, bath_labels

([0,
  150000,
  200000,
  250000,
  300000,
  350000,
  400000,
  450000,
  500000,
  600000,
  750000,
  1000000,
  2500000,
  5000000,
  10000000,
  76000001.0],
 [0, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 5000, 7500, 10000, 99991.0],
 [0, 0.11, 0.17, 0.23, 0.35, 0.5, 1, 1.5, 3, 10, 701.0],
 array([ 0.        ,  0.75      ,  1.        ,  1.10000002,  1.20000005,
         1.5       ,  1.75      ,  2.        ,  2.0999999 ,  2.20000005,
         2.5       ,  3.        ,  3.0999999 ,  3.5       ,  4.        ,
         4.0999999 ,  4.5       ,  5.        ,  6.        ,  7.        ,
         8.        ,  9.        , 10.        , 11.        , 12.        ,
        13.        , 14.        , 17.        , 23.        , 25.        ]),
 array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
        13., 16., 18.]))

In [647]:
# Dataset A: No binning at all.

STRING_FIELDS = ["State", "City", "Zipcode", "Street"]

data_a = pd.get_dummies(real_estate_data, columns=STRING_FIELDS)

In [648]:
# Dataset B: Use all bins.

data_b = real_estate_data.copy()

data_b["ListedPrice_Binned"] = pd.cut(
    data_b["ListedPrice"], bins=price_bins, labels=price_labels
)
data_b["Area_Binned"] = pd.cut(data_b["Area"], bins=area_bins, labels=area_labels)
data_b["Bedroom_Binned"] = pd.cut(data_b["Bedroom"], bins=bed_bins, labels=bed_labels)
data_b["Bathroom_Binned"] = pd.cut(
    data_b["Bathroom"], bins=bath_bins, labels=bath_labels
)
data_b["LotArea_Binned"] = pd.cut(data_b["LotArea"], bins=lot_bins, labels=lot_labels)

data_b = pd.get_dummies(
    data_b.drop(["ListedPrice", "Area", "Bathroom", "Bedroom", "LotArea"], axis=1),
    columns=STRING_FIELDS
    + [
        "ListedPrice_Binned",
        "Area_Binned",
        "Bedroom_Binned",
        "Bathroom_Binned",
        "LotArea_Binned",
    ],
)

In [649]:
# Dataset C: Area binned only.

data_c = real_estate_data.copy()

data_c["Area_Binned"] = pd.cut(data_c["Area"], bins=area_bins, labels=area_labels)
data_c["LotArea_Binned"] = pd.cut(data_c["LotArea"], bins=lot_bins, labels=lot_labels)

data_c = pd.get_dummies(
    data_c.drop(["Area", "LotArea"], axis=1),
    columns=STRING_FIELDS + ["Area_Binned", "LotArea_Binned"],
)

In [650]:
# Dataset D: Only price binned.

data_d = real_estate_data.copy()

data_d["ListedPrice_Binned"] = pd.cut(
    data_d["ListedPrice"], bins=price_bins, labels=price_labels
)

data_d = pd.get_dummies(
    data_d.drop(["ListedPrice"], axis=1), columns=STRING_FIELDS + ["ListedPrice_Binned"]
)

In [651]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from concurrent.futures import ThreadPoolExecutor, as_completed

# Train the decision tree on all of the datasets and compare the accuracy.


def train_and_evaluate(data: pd.DateOffset):
    X_train, X_test, y_train, y_test = train_test(data)

    model = DecisionTreeClassifier(random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy


print(f"Dataset A accuracy: {train_and_evaluate(data_a)}")
print(f"Dataset B accuracy: {train_and_evaluate(data_b)}")
print(f"Dataset C accuracy: {train_and_evaluate(data_c)}")
print(f"Dataset D accuracy: {train_and_evaluate(data_d)}")

Dataset A accuracy: 0.8216102963184675


ValueError: Cannot cast interval[float64, right] dtype to float32